In [1]:
import pandas as pd
import re
import nltk
from pymystem3 import Mystem
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

Задаем количество резюме в рекоментадельной выдаче.

In [2]:
TOP_COUNT = 30

Подгружаем текст из файла резюме ./data/vacancy/vacancy.html - этот файл заранее сохранен с сайта hh.ru.

In [3]:
with open('./data/vacancy/vacancy.html') as vacancy_page:
    soup = BeautifulSoup(vacancy_page, features="lxml")
    
def extract_vacancy_information(tag: str, attributes: dict):
    finder = soup.find_all(tag, attributes)
    if len(finder) == 1:
        finder = finder[0].get_text()
        return finder
    elif len(finder) > 1:
        listed_info = []
        for part in finder:
            list_element = part.get_text()
            listed_info.append(list_element)
    else:
        return "No information"
    
vacancy_description = extract_vacancy_information('div', {'data-qa': "vacancy-description"})

In [4]:
vacancy_description

'Вакансия открыта для отбора 2 кандидатов! Компания Fitness Нolding приглашает в команду проекта ZFitness на должность менеджера по продажам.Компания Fitness Нolding имеет репутацию надёжного и стабильного работодателя с 2003 года. Здесь вы найдёте возможность высокого заработка, карьерный рост, личное и профессиональное развитие, сплочённую и сильную команду, внимание и уважение со стороны коллег и руководителей, обучение и тренинги за счет компании.Требования к кандидату: - образование высшее, неоконченное высшее; - успешный опыт в продажах в сфере услуг; - нацеленность на высокий личный и командный результат; - базовые знания техник продаж и ведения переговоров; - готовность учиться и развиваться; - клиентоориентированность; - грамотная ,хорошо поставленная речь; - активность, стремление быть лучшим в продажах ; - готовность создавать и поддерживать высокий уровень сервиса; - знание ПК Word, Excel;   Обязанности: - входящая и исходящая работа по телефону(по теплой базе); - проведени

Подгружаем датафрейм с соискателями.

In [5]:
candidates = pd.read_csv('./data/merged/merged_file.csv')

In [6]:
df = candidates.copy()

In [7]:
df.head()

,Unnamed: 0,id,title,city,age,gender,area,desired_wage,work_experience,experience_description,education_level,languages,skills
0,0,4ad624ac0007f57b790039ed1f533448333337,менеджер по продажам,Воронеж,21,Мужчина,Строительство,35000,11.0,['Консультировании покупателей\n-активные прод...,Неоконченное высшее образование,"{'Русский': 'Родной', 'Английский': 'A2'}","['Энергичность', 'Стрессоустойчивость', 'Быстр..."
1,1,cfb865b70002ffc1360039ed1f704436587438,Специалист по продажам,Москва,51,Мужчина,Начало карьеры,137000,10.7,"['работа в СРМ, 1С, В2В,В2С, Поиск клиентов, П...",Высшее образование,"{'Русский': 'Родной', 'Английский': 'A1'}","['Ведение переговоров', 'Закупки', '1С: CRM ПР..."
2,2,8a7cdf1d0000749f6c0039ed1f736563726574,Менеджер по продажам,Москва,35,Мужчина,Автомобильный бизнес,100000,16.6,['Ведение сделки по продаже автомобиля от перв...,Высшее образование,"{'Русский': 'Родной', 'Английский': 'B2'}","['MS Office', 'MS Outlook', '1С: Предприятие 8..."
3,3,bc44cd50000178f56c0039ed1f3162566d664b,Менеджер по продажам,Москва,49,Мужчина,Продажи,120000,14.8,['-Планирование продаж\n-Определение и согласо...,Высшее образование,{'Russian': 'Native'},Сохраняю продуктивность даже в форс-мажорных с...
4,4,d9b39a3d000739766d0039ed1f6979784b5076,Менеджер по продаже запасных частей,Москва,48,Мужчина,Продажи,50000,18.6,"Опыт работы на платформе ABCP, 1С, работ с кат...",Образование,{'Russian': 'Native'},Большой опыт работы


Удаляем лишнюю для нас информацию и приводим оставшуюся к нижнему регистру. Описание опыта будет приведено к нижнему регистру на этапе лемматизации.

In [8]:
df.drop(['Unnamed: 0', 'title', 'education_level', 'languages', 'skills'], inplace=True, axis=1)
df['city'] = df['city'].apply(lambda x: x.lower())
df['gender'] = df['gender'].apply(lambda x: x.lower())
df['area'] = df['area'].apply(lambda x: x.lower())
df.head()

,id,city,age,gender,area,desired_wage,work_experience,experience_description
0,4ad624ac0007f57b790039ed1f533448333337,воронеж,21,мужчина,строительство,35000,11.0,['Консультировании покупателей\n-активные прод...
1,cfb865b70002ffc1360039ed1f704436587438,москва,51,мужчина,начало карьеры,137000,10.7,"['работа в СРМ, 1С, В2В,В2С, Поиск клиентов, П..."
2,8a7cdf1d0000749f6c0039ed1f736563726574,москва,35,мужчина,автомобильный бизнес,100000,16.6,['Ведение сделки по продаже автомобиля от перв...
3,bc44cd50000178f56c0039ed1f3162566d664b,москва,49,мужчина,продажи,120000,14.8,['-Планирование продаж\n-Определение и согласо...
4,d9b39a3d000739766d0039ed1f6979784b5076,москва,48,мужчина,продажи,50000,18.6,"Опыт работы на платформе ABCP, 1С, работ с кат..."


Лемматизируем описание опыта. Описание вакансии добавим в датафрейм последней строкой - это нужно для того, чтобы векторизация текста вакансии происходила одновременно с векторизацией текстов резюме и их размерность была одинакова. Также при этом векторизация будет учитывать частоту появления слов из описания вакансии во всем корпусе текста.

In [9]:
m = Mystem()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = ''.join(m.lemmatize(text.lower()))
    cleared_text = re.sub(r'[^а-яА-яёË]', ' ', lemm_text) 
    return ' '.join(cleared_text.split())

new_row = {'experience_description': vacancy_description}
df = df.append(new_row, ignore_index=True)
df['experience_description'] = df['experience_description'].apply(lemmatize_text)

Векторизируем текст посредством TfidfVectorizer.

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 3))
experience_description_vectorized = count_tf_idf.fit_transform(df['experience_description'].values.astype('U'))

[nltk_data] Downloading package stopwords to /Users/msher/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Вычисляем косинусное расстояние между векторами, полученными из текстов резюме и вектором, полученным из текста описания вакансии.

In [11]:
def count_cosine_similarity(row):
    return cosine_similarity(experience_description_vectorized[row.name],
                             experience_description_vectorized[experience_description_vectorized.shape[0] - 1])

In [12]:
df['cosine_similarity'] = df.apply(count_cosine_similarity, axis=1)

Выводим полученный датафрейм, отстортированный по убыванию cosine_similarity.

In [13]:
df.drop(index=experience_description_vectorized.shape[0] - 1, inplace=True)
df.sort_values(by = 'cosine_similarity', ascending=False).head(TOP_COUNT)

,id,city,age,gender,area,desired_wage,work_experience,experience_description,cosine_similarity
2279,bf4952af00053c0fd40039ed1f656a5a355373,москва,31.0,мужчина,спортивные клубы,90000.0,9.70,продажа клубный карта консультирование клиент ...,[[0.09992387819715616]]
3317,37f7c7a80002ebd3920039ed1f67484a765476,москва,28.0,женщина,продажи,150000.0,6.90,регулярный выполнение личный и коллективный пл...,[[0.09616674086762193]]
4627,f6aa1bb7000436f1ce0039ed1f4c4f58774142,москва,25.0,женщина,продажи,100000.0,6.60,обязанность общий руководство проект контроль ...,[[0.08190665600077443]]
2163,4c6057c60005696ba30039ed1f37554a587045,москва,28.0,мужчина,спортивные клубы,80000.0,5.20,встреча гость обработка входить поток работа с...,[[0.07162466362275566]]
1103,4c6b15c8000594255f0039ed1f6764724d6733,москва,37.0,мужчина,спортивные клубы,70000.0,10.11,консультация клиент по весь вид услуга клуб пп...,[[0.06551041222879808]]
4920,8dc100ff0003275ce20039ed1f556a6b4d4963,москва,24.0,женщина,спортивные клубы,60000.0,4.10,прием и качественный обработка входить заявка ...,[[0.06474280362061759]]
770,914487fa0006aa18980039ed1f5355774f5141,москва,24.0,женщина,банки,90000.0,3.80,работа с зарплатной клиент банк звонок встреча...,[[0.060367344569384765]]
2903,09031f330002fb81320039ed1f396f54697a69,москва,29.0,мужчина,продажи,200000.0,6.30,работа на этап строительство клуб продажа клуб...,[[0.05976038941410173]]
2684,85fd8e5a0004197a870039ed1f74335470706f,москва,24.0,женщина,маркетинг,80000.0,3.90,продажа клубный карта фитнес клуб консультация...,[[0.05938946414538623]]
1444,7694e177000204e2dd0039ed1f50434757584d,москва,25.0,мужчина,продажи,80000.0,4.50,прием и обработка входить заявка подбор трансп...,[[0.05864739721457391]]


Отфильтруем резюме кандидатов в исходном датафрейме по полученным индексам - это и будет наша финальная рекомендованная выборка.

In [14]:
candidates_sorted = candidates.iloc[df.sort_values(by = 'cosine_similarity', ascending=False).head(TOP_COUNT).index]
candidates_sorted

,Unnamed: 0,id,title,city,age,gender,area,desired_wage,work_experience,experience_description,education_level,languages,skills
2279,2279,bf4952af00053c0fd40039ed1f656a5a355373,Менеджер по продажам,Москва,31,Мужчина,Спортивные клубы,90000,9.70,['- Продажа клубных карт\n- Консультирование к...,Высшее образование,{'Russian': 'Native'},"['Опыт в сфере продаж', 'Знание методик активн..."
3317,3317,37f7c7a80002ebd3920039ed1f67484a765476,Менеджер по продажам,Москва,28,Женщина,Продажи,150000,6.90,['- регулярное выполнение личного и коллективн...,Высшее образование,"{'Русский': 'Родной', 'Английский': 'A1'}","['Работа в команде', 'Навыки продаж', 'Прямые ..."
4627,4627,f6aa1bb7000436f1ce0039ed1f4c4f58774142,Менеджер по продажам,Москва,25,Женщина,Продажи,100000,6.60,['Обязанности: \n1. Общее руководство проектом...,Образование,"{'Русский': 'Родной', 'Английский': 'B1'}",No information
2163,2163,4c6057c60005696ba30039ed1f37554a587045,Менеджер по продажам,Москва,28,Мужчина,Спортивные клубы,80000,5.20,"['Встреча гостей, обработка входящего потока, ...",Высшее образование,"{'Русский': 'Родной', 'Английский': 'A1'}","['Активные продажи', 'Консультирование клиенто..."
1103,1103,4c6b15c8000594255f0039ed1f6764724d6733,Менеджер по продажам,Москва,37,Мужчина,Спортивные клубы,70000,10.11,['Консультация клиентов по всем видам услуг кл...,Высшее образование,{'Russian': 'Native'},No information
4920,4920,8dc100ff0003275ce20039ed1f556a6b4d4963,Менеджер по работе с клиентами,Москва,24,Женщина,Спортивные клубы,60000,4.10,['Приём и качественная обработка входящих заяв...,Высшее образование (Бакалавр),"{'Русский': 'Родной', 'Английский': 'B2'}","['Умение работать в команде', 'Работа с кассой..."
770,770,914487fa0006aa18980039ed1f5355774f5141,Менеджер по продажам,Москва,24,Женщина,Банки,90000,3.80,['-работа с зарплатными клиентами банка (звонк...,Высшее образование,{'Russian': 'Native'},"['Холодные продажи', 'Поиск и привлечение клие..."
2903,2903,09031f330002fb81320039ed1f396f54697a69,РОП,Москва,29,Мужчина,Продажи,200000,6.30,"['-Работа на этапе строительства клуба, \n-Про...",Высшее образование,"{'Русский': 'Родной', 'Английский': 'B1'}","['Умение работать в команде', 'Прием и распред..."
2684,2684,85fd8e5a0004197a870039ed1f74335470706f,Менеджер по продажам услуг,Москва,24,Женщина,Маркетинг,80000,3.90,"['Продажа клубных карт фитнес клуба ', 'Консул...",Высшее образование,"{'Русский': 'Родной', 'Английский': 'A1'}","['Ответственность', 'Добросовестность', 'Креат..."
1444,1444,7694e177000204e2dd0039ed1f50434757584d,Менеджер по работе с клиентами,Москва,25,Мужчина,Продажи,80000,4.50,['— приём и обработка входящих заявок;\n— подб...,Высшее образование,"{'Русский': 'Родной', 'Английский': 'B1', 'Кур...","['Деловое общение', 'Внутренние коммуникации',..."
